In [38]:
import numpy
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D 
from keras.layers.convolutional import MaxPooling2D 
from keras.utils import np_utils
from keras import backend as K 
K.set_image_dim_ordering('th')

In [39]:
# fix random seed for reproducibility
seed = 7
       
numpy.random.seed(seed)

# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 

print('X_train.shape--',X_train.shape)
print('y_train.shape--',y_train.shape)
print('y_train[0]--',y_train[0])

print('X_test.shape--',X_test.shape)
print('y_test.shape--',y_test.shape)


('X_train.shape--', (50000, 3, 32, 32))
('y_train.shape--', (50000, 1))
('y_train[0]--', array([6], dtype=uint8))
('X_test.shape--', (10000, 3, 32, 32))
('y_test.shape--', (10000, 1))


In [40]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print('y_test[0]--',y_test[0])
num_classes = y_test.shape[1]

('y_test[0]--', array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]))


In [41]:
# Create the model
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, 32, 32), border_mode='same',activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same',W_constraint=maxnorm(3))) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', W_constraint=maxnorm(3))) 
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')) # Compile model

#compile model
epochs = 10
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False) 
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy']) 
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_5 (Convolution2D)  (None, 32, 32, 32)    896         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 32, 32, 32)    0           convolution2d_5[0][0]            
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 32, 32, 32)    9248        dropout_5[0][0]                  
____________________________________________________________________________________________________
maxpooling2d_3 (MaxPooling2D)    (None, 32, 16, 16)    0           convolution2d_6[0][0]            
___________________________________________________________________________________________

In [42]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          nb_epoch=epochs,batch_size=16, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
1200s - loss: 14.5026 - acc: 0.1002 - val_loss: 14.5079 - val_acc: 0.0999
Epoch 2/10
1164s - loss: 14.5079 - acc: 0.0999 - val_loss: 14.5079 - val_acc: 0.0999
Epoch 3/10
1077s - loss: 14.5114 - acc: 0.0997 - val_loss: 14.5079 - val_acc: 0.0999
Epoch 4/10
1104s - loss: 14.4966 - acc: 0.1006 - val_loss: 14.5079 - val_acc: 0.0999
Epoch 5/10
1142s - loss: 14.5027 - acc: 0.1002 - val_loss: 14.5079 - val_acc: 0.0999
Epoch 6/10
1086s - loss: 14.5018 - acc: 0.1003 - val_loss: 14.5079 - val_acc: 0.0999
Epoch 7/10
1090s - loss: 14.5092 - acc: 0.0998 - val_loss: 14.5079 - val_acc: 0.0999
Epoch 8/10
1017s - loss: 14.5060 - acc: 0.1000 - val_loss: 14.5079 - val_acc: 0.0999
Epoch 9/10
1075s - loss: 14.4908 - acc: 0.1010 - val_loss: 14.5079 - val_acc: 0.0999
Epoch 10/10
984s - loss: 14.5214 - acc: 0.0991 - val_loss: 14.5079 - val_acc: 0.0999
Accuracy: 9.99%
